## Neural networks
## Tutorial for implementing *KerasNN* class

When applying Tensorflow and Keras to strengthen the understanding of how neural networks operate, a class was developed to instantiate models based on Keras API of Tensorflow models. Its use is pretty straightforward, since it requires only the definition of the architecture and main hyper-parameters of the neural network. This simplifies even further the construction of Tensorflow/Keras models.
<br>
<br>
All estimations have no intention of being as efficient as possibile, but focus on illustrating how this class can be used in real-world applications.

--------

This notebook imports the developed class in addition to presenting a data pre-processing pipeline that seeks to assess its functionalities by fitting a feedforward neural network to a binary classification problem.

**Summary:**
1. [Libraries](#libraries)<a href='#libraries'></a>.
2. [Functions and classes](#functions_classes)<a href='#functions_classes'></a>.
3. [Settings](#settings)<a href='#settings'></a>.
4. [Importing datasets](#imports)<a href='#imports'></a>.
5. [Data pre-processing](#data_pre_proc)<a href='#data_pre_proc'></a>.
6. [Feedforward neural network](#feedforward_nn)<a href='#feedforward_nn'></a>.
    * [Settings](#nn_settings)<a href='#nn_settings'></a>.
    * [Model fitting](#model_fitting)<a href='#model_fitting'></a>.

<a id='libraries'></a>

## Libraries

In [1]:
import pandas as pd
import numpy as np
import json
import os

from datetime import datetime
import time
import progressbar

from sklearn.metrics import roc_auc_score, average_precision_score, auc, precision_recall_curve, brier_score_loss

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

<a id='functions_classes'></a>

## Functions and classes

In [2]:
import utils
from utils import loading_data, classify_features, assessing_missings
from utils import data_consistency, running_time, missings_detection, frequency_list

In [3]:
import transformations
from transformations import log_transformation, standard_scale, impute_missing, one_hot_encoding
from transformations import applying_log_transf, applying_standard_scale, treating_missings, applying_one_hot

In [4]:
import keras_nn
from keras_nn import KerasNN, PerformanceHistory, EarlyStoppingROC

<a id='settings'></a>

## Settings

In [5]:
# Declare whether to export results:
export = True

# Declare whether to log-transform numerical features:
log_transform = True

# Declare whether to standardize numerical features:
standardize = True

# Define the dataset_id:
dataset_id = 2706

<a id='imports'></a>

## Importing datasets

<a id='feats_label'></a>

### Features and label

In [6]:
print('----------------------------------------')
print(f'\033[1mStore {dataset_id}:\033[0m')

df_train = loading_data(path=f'../../data_pipeline/Datasets/dataset_{dataset_id}.csv',
                        dtype={'order_id': str, 'store_id': int, 'epoch': str},
                        id_var='order_id')

print('----------------------------------------')
print('\n')

# Accessory variables:
drop_vars = ['y', 'order_amount', 'store_id', 'order_id', 'status', 'epoch', 'date', 'weight']

df_train.head(3)

----------------------------------------
Store 2706:
Shape of df: (14434, 2628).
Number of distinct instances: 14434.
Time period: from 2020-12-31 to 2021-03-31.
----------------------------------------




,AVGITEMCREATIONTIME(),BILLINGADDRESSCHARRANDOMNESS(),BILLINGADDRESSCHARWORDMODELPROB(),BILLINGADDRESSRANDOMNESS(),BILLINGCITY(),BILLINGCOUNTRY(),BILLINGLARGEAREAREPUTATION(),BILLINGNAMECHARRANDOMNESS(),BILLINGNAMECHARWORDMODELPROB(),BILLINGNAMERANDOMNESS(),...,ZIPFIRST3REPUTATION(),ZIPFIRST5REPUTATION(),y,order_amount,order_id,status,epoch,store_id,weight,date
0,NaN,0.255415,0.194991,9.357623e-14,Venustiano carranza,MX,NaN,0.983040,0.034951,7.736941e-01,...,0.061190,0.045009,0.0,1795.16,130874044224,APPROVED,1609459270000.0,2706,1.0,2020-12-31 21:01:10
1,NaN,0.299488,0.117108,9.357623e-14,Coacalco de berriozabal,MX,NaN,0.351049,0.097665,2.301321e-09,...,0.091155,0.078335,0.0,4092.23,130874049768,APPROVED,1609459766000.0,2706,1.0,2020-12-31 21:09:26
2,NaN,0.201846,0.204315,9.357623e-14,Hermosillo,MX,NaN,0.189533,0.180326,9.357623e-14,...,0.069444,0.118511,0.0,1871.40,130874053609,APPROVED,1609460116000.0,2706,1.0,2020-12-31 21:15:16


#### Train-validation-test split

In [7]:
df_train['train_val_test'] = 'test'
df_train['train_val_test'].iloc[int(df_train.shape[0]/3): int((2*df_train.shape[0])/3)] = 'val'
df_train['train_val_test'].iloc[:int(df_train.shape[0]/3)] = 'train'

# Train-test split:
df_test = df_train[df_train.train_val_test == 'test'].copy()
df_val = df_train[df_train.train_val_test == 'val'].copy()
df_train = df_train[df_train.train_val_test == 'train'].copy()

# Resetting indices:
df_train.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

drop_vars.append('train_val_test')

<a id='classif_feat'></a>

### Classifying features

In [8]:
print(f'\033[1mDataset {dataset_id}:\033[0m')
classified_features = classify_features(df_train, drop_vars=drop_vars,
                                        drop_excessive_miss=True, drop_no_var=True,
                                        validation_data=df_val,
                                        test_data=df_test)

feats_assess = classified_features['feats_assess']
cat_vars = classified_features['cat_vars']
excessive_miss_train = classified_features['excessive_miss_train']
no_variance = classified_features['no_variance']
cont_vars = classified_features['cont_vars']
binary_vars = classified_features['binary_vars']
    
feats_assess

Dataset 2706:
Initial number of features: 2620.
1377 features were dropped for excessive number of missings!
303 features were dropped for having no variance!
940 remaining features.




,class,frequency
2,cont_vars,912
0,cat_vars,14
1,binary_vars,14
3,drop_vars,9


<a id='data_pre_proc'></a>

## Data pre-processing

<a id='assessing_missing'></a>

### Assessing missing values

In [9]:
print(f'\033[1mDataset {dataset_id}:\033[0m')
print('\033[1mTraining data:\033[0m')
missings_train = assessing_missings(dataframe=df_train)
print('\n\033[1mValidation data:\033[0m')
missings_val = assessing_missings(dataframe=df_val)
print('\n\033[1mTest data:\033[0m')
missings_test = assessing_missings(dataframe=df_test)
print('\n')

missings_train.index.name = 'training_data'
missings_val.index.name = 'validation_data'
missings_test.index.name = 'test_data'

missings_train.head(10)

Dataset 2706:
Training data:
Number of features with missings: 230 out of 949 features (24.24%).
Average number of missings: 317 out of 4811 observations (6.59%).

Validation data:
Number of features with missings: 134 out of 949 features (14.12%).
Average number of missings: 320 out of 4811 observations (6.65%).

Test data:
Number of features with missings: 133 out of 949 features (14.01%).
Average number of missings: 312 out of 4812 observations (6.48%).




,feature,missings,share
training_data,,,
0,"CUSTNAVCOUNT(t,30min)",4488,0.932862
1,"CUSTNAVCOUNT(t,1h)",4475,0.930160
2,CUSTNAVENTROPY(COOKIE),4471,0.929329
3,"CUSTNAVCOUNT(9,30min)",4470,0.929121
4,"CUSTNAVCOUNT(9,1h)",4450,0.924964
5,"CUSTNAVCOUNT(t,1d)",4433,0.921430
6,"CUSTNAVCOUNT(ta,30min)",4428,0.920391
7,"CUSTNAVCOUNT(ta,1h)",4408,0.916234
8,"CREDITCARD(TOTAL_AMOUNT,60)",4376,0.909582


<a id='num_transf'></a>

### Transforming numerical features

#### Logarithmic transformation

In [10]:
print('---------------------------------------------------------------------------------------------------------')
print('\033[1mAPPLYING LOGARITHMIC TRANSFORMATION OVER NUMERICAL DATA\033[0m')
print('\n')
print(f'\033[1mDataset {dataset_id}:\033[0m')

# Variables that should not be log-transformed:
not_log = [c for c in df_train.columns if c not in cont_vars]

if log_transform:
    print('\033[1mTraining data:\033[0m')
    df_train = applying_log_transf(dataframe=df_train, not_log=not_log)
    
    print('\033[1mValidation data:\033[0m')
    df_val = applying_log_transf(dataframe=df_val, not_log=not_log)

    print('\033[1mTest data:\033[0m')
    df_test = applying_log_transf(dataframe=df_test, not_log=not_log)
    print('\n')


else:
    print('\033[1mNo transformation performed!\033[0m')
    print('\n')

print('---------------------------------------------------------------------------------------------------------')
print('\n')

---------------------------------------------------------------------------------------------------------
APPLYING LOGARITHMIC TRANSFORMATION OVER NUMERICAL DATA


Dataset 2706:
Training data:
Number of numerical variables log-transformed: 912.
Validation data:
Number of numerical variables log-transformed: 912.
Test data:
Number of numerical variables log-transformed: 912.


---------------------------------------------------------------------------------------------------------




#### Standardizing numerical features

In [11]:
print('---------------------------------------------------------------------------------------------------------')
print('\033[1mAPPLYING STANDARD SCALE TRANSFORMATION OVER NUMERICAL DATA\033[0m')
print('\n')

print(f'\033[1mDataset {dataset_id}:\033[0m')

# Inputs that should not be standardized:
not_stand = [c for c in df_train.columns if c.replace('L#', '') not in cont_vars]

if standardize:
    scaled_data = applying_standard_scale(training_data=df_train, not_stand=not_stand,
                                          validation_data=df_val, test_data=df_test)
    df_train_scaled = scaled_data['training_data']
    df_val_scaled = scaled_data['validation_data']
    df_test_scaled = scaled_data['test_data']

else:
    df_train_scaled = df_train.copy()
    df_val_scaled = df_val.copy()
    df_test_scaled = df_test.copy()

    print('\033[1mNo transformation performed!\033[0m')

print('\n')
print('---------------------------------------------------------------------------------------------------------')
print('\n')

---------------------------------------------------------------------------------------------------------
APPLYING STANDARD SCALE TRANSFORMATION OVER NUMERICAL DATA


Dataset 2706:
Standard scaling training data...
Standard scaling validation data...
Standard scaling test data...


---------------------------------------------------------------------------------------------------------




In [12]:
del scaled_data

#### Treating missing values

In [13]:
print('---------------------------------------------------------------------------------------------------------')
print('\033[1mTREATING MISSING VALUES\033[0m')
print('\n')

print(f'\033[1mDataset {dataset_id}:\033[0m')

print('\033[1mTreating missing values of training data...\033[0m')
df_train_scaled = treating_missings(dataframe=df_train_scaled, cat_vars=cat_vars,
                                       drop_vars=drop_vars)

print('\033[1mTreating missing values of validation data...\033[0m')
df_val_scaled = treating_missings(dataframe=df_val_scaled, cat_vars=cat_vars,
                                       drop_vars=drop_vars)

print('\033[1mTreating missing values of test data...\033[0m')
df_test_scaled = treating_missings(dataframe=df_test_scaled, cat_vars=cat_vars,
                                   drop_vars=drop_vars)

print('\n')
print('---------------------------------------------------------------------------------------------------------')
print('\n')

---------------------------------------------------------------------------------------------------------
TREATING MISSING VALUES


Dataset 2706:
Treating missing values of training data...
Treating missing values of validation data...
Treating missing values of test data...


---------------------------------------------------------------------------------------------------------




<a id='categorical_transf'></a>

### Transforming categorical features

#### Creating dummies through one-hot encoding

In [14]:
print(f'\033[1mDataset {dataset_id}:\033[0m')

transf_data = applying_one_hot(df_train_scaled, cat_vars, validation_data=df_val_scaled, test_data=df_test_scaled)
df_train_scaled = transf_data['training_data']
df_val_scaled = transf_data['validation_data']
df_test_scaled = transf_data['test_data']

print(f'\033[1mShape of df_train_scaled:\033[0m {df_train_scaled.shape}.')
print(f'\033[1mShape of df_val_scaled:\033[0m {df_val_scaled.shape}.')
print(f'\033[1mShape of df_test_scaled:\033[0m {df_test_scaled.shape}.')
print('\n')

Dataset 2706:
Number of categorical features: 14
Number of overall selected dummies: 125.
Shape of df_train_scaled: (4811, 1286).
Shape of df_val_scaled: (4811, 1190).
Shape of df_test_scaled: (4812, 1189).




In [15]:
del transf_data

In [16]:
# Assessing missing values (training data):
missings_detection(df_train_scaled, name=f'df_train_scaled (dataset {dataset_id})')

# Assessing missing values (test data):
missings_detection(df_val_scaled, name=f'df_val_scaled (dataset {dataset_id})')

# Assessing missing values (test data):
missings_detection(df_test_scaled, name=f'df_test_scaled (dataset {dataset_id})')

<a id='datasets_structure'></a>

### Datasets structure

In [17]:
print(f'\033[1mDataset {dataset_id}\033[0m:')
consistent_data = data_consistency(dataframe=df_train_scaled, validation_data=df_val_scaled,
                                   test_data=df_test_scaled)

df_val_scaled = consistent_data['validation_data']
df_test_scaled = consistent_data['test_data']

Dataset 2706:
Training and validation data are consistent with each other.
Training and test data are consistent with each other.


<a id='feedforward_nn'></a>

## Feedforward neural network

In [18]:
# Converting data from dataframes into nd-arrays:
X_train = df_train_scaled.drop(drop_vars, axis=1).values
y_train = df_train_scaled['y'].values

X_val = df_val_scaled.drop(drop_vars, axis=1).values
y_val = df_val_scaled['y'].values

X_test = df_test_scaled.drop(drop_vars, axis=1).values
y_test = df_test_scaled['y'].values

<a id='nn_settings'></a>

### Settings

In [19]:
# Model architecture:
model_architecture = {1: {'neurons': int(np.floor((X_train.shape[1] + 1)*0.1)),
                          'activation': 'tanh',
                          'dropout_param': 0.1}}

# Functions:
output_activation = 'sigmoid'
cost_function = 'binary_crossentropy'

# Hyper-parameters:
num_epochs = 200
batch_size = 512
es_param = None
regularization = 'l2'
regul_param = 1e-06
input_dropout = 0
weights_init = 'glorot_uniform'

# Defining the optimizer:
default_adam = False
optimizer = 'adam'
opt_params = {
    'learning_rate': 0.001,
    'beta_1': 0.5,
    'beta_2': 0.9999,
    'epsilon': 1e-07
}

<a id='model_fitting'></a>

### Model fitting

In [20]:
nn_start_time = datetime.now()

# Creating neural network object, declaring its architecture and defining hyper-parameters:
model = KerasNN(model_architecture = model_architecture, num_inputs = X_train.shape[1],
                output_activation = output_activation, cost_function = cost_function,
                num_epochs = num_epochs, batch_size = batch_size,
                default_adam = default_adam, optimizer = optimizer, opt_params = opt_params,
                regularization = regularization, regul_param = regul_param,
                input_dropout = input_dropout,
                weights_init = weights_init)

# Training the model:
model.run(train_inputs = X_train, train_output = y_train,
          val_inputs = X_val, val_output = y_val,
          early_stopping = False,
          verbose = 1)

# Performance metrics on validation data:
val_roc_auc = roc_auc_score(y_val, [p[0] for p in model.predictions])
val_avg_prec_score = average_precision_score(y_val, [p[0] for p in model.predictions])
val_brier_score = brier_score_loss(y_val, [p[0] for p in model.predictions])

# Cost function by training epoch:
model_costs = model.model_costs

# Information relating cost function:
min_cost = model_costs.loss.min()
epoch_min_cost = model_costs.loss.idxmin() + 1
val_min_cost = model_costs.val_loss.min()
epoch_min_cost = model_costs.val_loss.idxmin() + 1

# Assessing running time:
nn_end_time = datetime.now()

nn_time = running_time(start_time=nn_start_time, end_time=nn_end_time)

Epoch 1/200
10/10 [==============================] - 1s 58ms/step - loss: 0.2585 - val_loss: 0.0957
Epoch 2/200
10/10 [==============================] - 0s 39ms/step - loss: 0.0778 - val_loss: 0.0752
Epoch 3/200
10/10 [==============================] - 0s 38ms/step - loss: 0.0566 - val_loss: 0.0676
Epoch 4/200
10/10 [==============================] - 0s 39ms/step - loss: 0.0466 - val_loss: 0.0633
Epoch 5/200
10/10 [==============================] - 0s 38ms/step - loss: 0.0390 - val_loss: 0.0604
Epoch 6/200
10/10 [==============================] - 0s 38ms/step - loss: 0.0351 - val_loss: 0.0592
Epoch 7/200
10/10 [==============================] - 0s 43ms/step - loss: 0.0307 - val_loss: 0.0587
Epoch 8/200
10/10 [==============================] - 0s 38ms/step - loss: 0.0280 - val_loss: 0.0581
Epoch 9/200
10/10 [==============================] - 0s 38ms/step - loss: 0.0256 - val_loss: 0.0578
Epoch 10/200
10/10 [==============================] - 0s 38ms/step - loss: 0.0228 - val_loss: 0.0576

10/10 [==============================] - 0s 38ms/step - loss: 9.2431e-04 - val_loss: 0.0756
Epoch 83/200
10/10 [==============================] - 0s 38ms/step - loss: 9.3356e-04 - val_loss: 0.0760
Epoch 84/200
10/10 [==============================] - 0s 38ms/step - loss: 8.6692e-04 - val_loss: 0.0765
Epoch 85/200
10/10 [==============================] - 0s 37ms/step - loss: 8.6823e-04 - val_loss: 0.0768
Epoch 86/200
10/10 [==============================] - 0s 38ms/step - loss: 8.5218e-04 - val_loss: 0.0766
Epoch 87/200
10/10 [==============================] - 0s 38ms/step - loss: 8.4807e-04 - val_loss: 0.0770
Epoch 88/200
10/10 [==============================] - 0s 38ms/step - loss: 0.0010 - val_loss: 0.0771
Epoch 89/200
10/10 [==============================] - 0s 38ms/step - loss: 8.6194e-04 - val_loss: 0.0773
Epoch 90/200
10/10 [==============================] - 0s 39ms/step - loss: 8.6270e-04 - val_loss: 0.0772
Epoch 91/200
10/10 [==============================] - 0s 47ms/step - los

10/10 [==============================] - 0s 40ms/step - loss: 4.5611e-04 - val_loss: 0.0846
Epoch 160/200
10/10 [==============================] - 0s 40ms/step - loss: 4.6968e-04 - val_loss: 0.0846
Epoch 161/200
10/10 [==============================] - 1s 132ms/step - loss: 4.8193e-04 - val_loss: 0.0848
Epoch 162/200
10/10 [==============================] - 0s 38ms/step - loss: 4.5996e-04 - val_loss: 0.0848
Epoch 163/200
10/10 [==============================] - 0s 39ms/step - loss: 5.2816e-04 - val_loss: 0.0848
Epoch 164/200
10/10 [==============================] - 0s 37ms/step - loss: 4.7380e-04 - val_loss: 0.0845
Epoch 165/200
10/10 [==============================] - 0s 39ms/step - loss: 4.6671e-04 - val_loss: 0.0847
Epoch 166/200
10/10 [==============================] - 0s 39ms/step - loss: 4.8495e-04 - val_loss: 0.0849
Epoch 167/200
10/10 [==============================] - 0s 38ms/step - loss: 4.6060e-04 - val_loss: 0.0850
Epoch 168/200
10/10 [==============================] - 0s 3

In [21]:
print(f'Validation ROC-AUC: {val_roc_auc:.4f}.')
print(f'Validation average precision score: {val_avg_prec_score:.4f}.')
print(f'Validation Brier score: {val_brier_score:.4f}.')

Validation ROC-AUC: 0.9716.
Validation average precision score: 0.8577.
Validation Brier score: 0.0131.


In [22]:
# Create figure:
fig = make_subplots(specs=[[{'secondary_y': True}]])

# Create the plot (first axis):
fig.add_trace(
    go.Scatter(x=[i+1 for i in range(len(model.epoch_performance['epoch_val_roc_auc']))],
               y=model.epoch_performance['epoch_val_roc_auc'], name='Val ROC-AUC',
               hovertemplate =
                'ROC-AUC = %{y:.4f}<br>'+
                'epoch = %{x}<br>'
              ),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=[i+1 for i in range(len(model.epoch_performance['epoch_val_avg_prec_score']))],
               y=model.epoch_performance['epoch_val_avg_prec_score'], name='Val avg precision',
               hovertemplate = 'Avg precision = %{y:.4f}<br>'+
                               'epoch = %{x}<br>',
               marker_color='orange',
               mode='lines'
              ),
    secondary_y=True,
)

# Changing layout:
fig.update_layout(
    title_text='Validation performance by epoch of training<br>',
    width=700,
    height=400
)

# Set labels:
fig.update_xaxes(title_text='epoch')
fig.update_yaxes(title_text='performance', secondary_y=False)

fig.show()

<a id='model_fitting_es'></a>

### Model fitting with early stopping

In [23]:
es_param = {'min_delta': 0.0001, 'patience': 10, 'consecutive_patience': False}

In [24]:
nn_start_time = datetime.now()

# Creating neural network object, declaring its architecture and defining hyper-parameters:
model = KerasNN(model_architecture = model_architecture, num_inputs = X_train.shape[1],
                 output_activation = output_activation, cost_function = cost_function,
                 num_epochs = num_epochs, batch_size = batch_size,
                 default_adam = default_adam, optimizer = optimizer, opt_params = opt_params,
                 regularization = regularization, regul_param = regul_param,
                 input_dropout = input_dropout,
                 weights_init = weights_init)

# Training the model:
model.run(train_inputs = X_train, train_output = y_train,
          val_inputs = X_val, val_output = y_val,
          test_inputs = X_test, test_output = y_test,
          early_stopping = True, es_params = es_param,
          verbose = 1)

# Performance metrics on validation data:
test_roc_auc = roc_auc_score(y_test, [p[0] for p in model.predictions['test']])
test_avg_prec_score = average_precision_score(y_test, [p[0] for p in model.predictions['test']])
test_brier_score = brier_score_loss(y_test, [p[0] for p in model.predictions['test']])

# Cost function by training epoch:
model_costs = model.model_costs

# Information relating cost function:
min_cost = model_costs.loss.min()
epoch_min_cost = model_costs.loss.idxmin() + 1
val_min_cost = model_costs.val_loss.min()
epoch_min_cost = model_costs.val_loss.idxmin() + 1

# Assessing running time:
nn_end_time = datetime.now()

nn_time = running_time(start_time=nn_start_time, end_time=nn_end_time)

Epoch 1/200
10/10 [==============================] - 1s 56ms/step - loss: 0.2593 - val_loss: 0.0893
Epoch 2/200
10/10 [==============================] - 0s 47ms/step - loss: 0.0749 - val_loss: 0.0711
Epoch 3/200
10/10 [==============================] - 0s 49ms/step - loss: 0.0561 - val_loss: 0.0634
Epoch 4/200
10/10 [==============================] - 0s 47ms/step - loss: 0.0464 - val_loss: 0.0600
Epoch 5/200
10/10 [==============================] - 0s 47ms/step - loss: 0.0398 - val_loss: 0.0588
Epoch 6/200
10/10 [==============================] - 0s 47ms/step - loss: 0.0345 - val_loss: 0.0566
Epoch 7/200
10/10 [==============================] - 0s 47ms/step - loss: 0.0316 - val_loss: 0.0565
Epoch 8/200
10/10 [==============================] - 0s 46ms/step - loss: 0.0282 - val_loss: 0.0557
Epoch 9/200
10/10 [==============================] - 0s 47ms/step - loss: 0.0252 - val_loss: 0.0554
Epoch 10/200
10/10 [==============================] - 0s 48ms/step - loss: 0.0227 - val_loss: 0.0551

In [25]:
print(f'Test ROC-AUC: {test_roc_auc:.4f}.')
print(f'Test average precision score: {test_avg_prec_score:.4f}.')
print(f'Test Brier score: {test_brier_score:.4f}.')

Test ROC-AUC: 0.9809.
Test average precision score: 0.8761.
Test Brier score: 0.0124.
